In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
prices = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_prices.parquet')

In [3]:
prices.head()

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
0,200102.0,808.0,210101,12.0,0.35,Q,210101
1,199603.0,32.0,210101,3.0,0.42,,210101
2,198905.0,3.0,210101,8.0,0.44,,210101
3,199511.0,52.0,210101,2.0,0.64,,210101
4,200105.0,126.0,210101,8.0,0.80,,210101


In [4]:
items = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_item.parquet')
items.head()

,item_id,description,date_quote_s,date_quote_e,n_obs
0,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
2,210105,LARGE WHOLEMEAL LOAF-UNSLICED,198802,200301,27161
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361


In [18]:
items[items["description"].str.contains("RIce", case = False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
15,210202,RICE-LONG GRAIN-WHITE-500G,198802,200301,27513
23,210212,BASMATI RICE 500G-1KG,200302,202510,35900
28,210217,RICE MICRO POUCH/TRAY 220-280G,201602,202510,27087
32,210222,RICE CAKES PACK 100-180G,202402,202510,3480
1269,630226,RECORDABLE CD PRICE PER CD,200502,202510,51758
1304,630345,GOLF BALLS PRICE PER BALL,200402,202510,31262
1386,640406,HOTEL 1 NIGHT PRICE,200502,202510,27407


In [6]:
df = prices.merge(items[["item_id", "description"]], on="item_id", how="left")
df.head()

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id,description
0,200102.0,808.0,210101,12.0,0.35,Q,210101,LARGE LOAF-WHITE-SLICED-800G
1,199603.0,32.0,210101,3.0,0.42,,210101,LARGE LOAF-WHITE-SLICED-800G
2,198905.0,3.0,210101,8.0,0.44,,210101,LARGE LOAF-WHITE-SLICED-800G
3,199511.0,52.0,210101,2.0,0.64,,210101,LARGE LOAF-WHITE-SLICED-800G
4,200105.0,126.0,210101,8.0,0.80,,210101,LARGE LOAF-WHITE-SLICED-800G


In [19]:
ids = [210101, 210202, 211701, 211602, 210905, 210903, 210907, 210910, 210102, 210105, 210106, 210107, 211603, 211604, 211714, 211712, 211709, 211711, 211710, 210212, 210217, 210222]

df_filtered = df[df["item_id"].isin(ids)]

In [20]:
df_filtered['quote_date'] = pd.to_datetime(df_filtered['quote_date'], format='%Y%m')
df_filtered['year'] = df_filtered['quote_date'].dt.year
df_filtered['month'] = df_filtered['quote_date'].dt.month

/tmp/ipython-input-1736550247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['quote_date'] = pd.to_datetime(df_filtered['quote_date'], format='%Y%m')
/tmp/ipython-input-1736550247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['year'] = df_filtered['quote_date'].dt.year
/tmp/ipython-input-1736550247.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [21]:
df_filtered.groupby(["item_id", "year"])["price"].mean()

item_id  year
210101   1988    0.479650
         1989    0.504032
         1990    0.521996
         1991    0.556271
         1992    0.575522
                   ...   
211714   2021    1.397954
         2022    1.528745
         2023    1.752900
         2024    1.728006
         2025    1.766029
Name: price, Length: 442, dtype: float32

In [22]:
def categorize_product(desc):
    desc_lower = desc.lower()
    if 'milk' in desc_lower:
        return 'Milk'
    elif 'bread' in desc_lower:
        return 'Bread'
    elif 'eggs' in desc_lower or 'egg' in desc_lower:
        return 'Eggs'
    elif 'rice' in desc_lower:
        return 'Rice'
    elif 'chicken' in desc_lower:
        return 'Chicken'
    else:
        return 'Other'

df_filtered['product_category'] = df_filtered['description'].apply(categorize_product)

/tmp/ipython-input-3587648725.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['product_category'] = df_filtered['description'].apply(categorize_product)


In [23]:
heatmap_data = df_filtered.groupby(['product_category', 'year'])['price'].mean().reset_index()
heatmap_data.columns = ['product', 'year', 'avg_price']
heatmap_data['avg_price'] = heatmap_data['avg_price'].round(2)
heatmap_data = heatmap_data[(heatmap_data['year'] >= 2000) & (heatmap_data['year'] <= 2025)]


In [25]:
heatmap_data.to_csv('uk_prices_heatmap.csv', index=False)

In [24]:
heatmap_data

,product,year,avg_price
12,Bread,2000,0.76
13,Bread,2001,0.77
14,Bread,2002,0.83
15,Bread,2003,0.86
16,Bread,2004,0.81
...,...,...,...
223,Rice,2021,1.25
224,Rice,2022,1.30
225,Rice,2023,1.40
226,Rice,2024,1.38
